In [ ]:
import os
from pathlib import Path
from PIL import Image
import source.llm_input as llm_i
import source.llm_output as llm_o

In [ ]:
os.getcwd()

## Test convert_tif_to_jpg

### Define file paths:

In [ ]:
script_dir = os.getcwd()
source_folder = os.path.join(script_dir, '..', 'test_llm_input/data_tif_to_jpg')
destination_folder = os.path.join(script_dir, '..', 'test_llm_input/data_jpg')
destination_folder

### Check files in destination folder:

In [ ]:
file_list = os.listdir(destination_folder)
file_list.sort()
file_list

In [ ]:
# Check file format and whether it is either jpeg or tiff:
for file in file_list:
    file_path = os.path.join(destination_folder, file)
    image = Image.open(file_path)
    actual_format = image.format.lower()  # Returns 'JPEG', 'TIFF', etc.
    print(actual_format)
    acceptable_format =  actual_format not in ['jpeg', 'tiff']
    print(acceptable_format)

### Execute function:

In [ ]:
llm_i.convert_tif_to_jpg(source_folder, destination_folder, quality=85)

### Check files in destination folder:

In [ ]:
file_list = os.listdir(destination_folder)
file_list.sort()
file_list

### Remove newly generated files to return to original state:

## Test i.convert_image_if_needed

### Define file paths:

In [ ]:
file_conv_folder = os.path.join(script_dir, '..', 'test_llm_input/data_tif_jpg_conv')

file_path = os.path.join(file_conv_folder, 'BernerOberland001.tif')


### Execute function:

In [ ]:
file_list = os.listdir(file_conv_folder)
file_list.sort()
for file in file_list:
    file_path = os.path.join(file_conv_folder, file)
    file_path_conv = llm_i.convert_image_if_needed(file_path)
    print(file_path_conv)

In [ ]:
file_list = os.listdir(file_conv_folder)
file_list.sort()
file_list

### Remove the newly generated files to return to the original state:

In [ ]:
file_list = os.listdir(file_conv_folder)

for file in file_list:
    file_path = os.path.join(file_conv_folder, file)
    #print(file_path)
    #print(file.split('.'))
    name_part = file.split('.')[0]
    if name_part.split('_')[-1] == 'converted':
        print(name_part)
        os.remove(file_path)


## Test parse_response_to_dict:

In [ ]:
jpg_data_path = os.path.join(script_dir, '..', 'test_llm_input/data_jpg')

In [ ]:
example_file = os.listdir(jpg_data_path)[0]
example_path = os.path.join(jpg_data_path, example_file)
example_path

In [ ]:
prompt = 'Analyze this image. If the image contains one or multiple houses, say yes. Say no, otherwise.'


In [ ]:
llm_output = llm_i.call_ollama_model(example_path, prompt)
llm_output

In [ ]:
def create_analysis_prompt_basic():
    """Create the structured prompt for image analysis."""
    return """
    Analyze this image and return ONLY a Python dictionary in exactly this format:
    
    {
        'image_is_photography': X,  # True if the image is a photography, False otherwise
        'additional_comments': '' # Any additional observations or empty string if none
    }
    
    Replace X with True (image is a photography) or False (image is not a photography).
    Return ONLY the dictionary, no other text.
    Your answer MUST have the exact structue of the dictionary described above (all keys MUST be present). 
    If you cannot answer the question in the way implied by this structure, enter 'None' as value and offer 
    your answer and explanations under 'additional_comments'.
    """

In [ ]:
prompt = create_analysis_prompt_basic()

In [ ]:
llm_output = llm_i.call_ollama_model(example_path, prompt)
llm_output

In [ ]:
parsed_output = llm_o.parse_response_to_dict(llm_output)
parsed_output

In [ ]:
structured_text = "{'image_is_photography': True, 'additional_comments': 'no comment'}"


In [ ]:
semi_structured_text = "OK, I don't like dictionaries but here is one: {'image_is_photography': False, 'additional_comments': ''}"


In [ ]:
unstructured_text = "hey, this is so funny! Hahaha!"

In [ ]:
parsed_output = llm_o.parse_response_to_dict(structured_text)
parsed_output

In [ ]:
parsed_output = llm_o.parse_response_to_dict(semi_structured_text)
parsed_output

In [ ]:
parsed_output = llm_o.parse_response_to_dict(unstructured_text)
parsed_output

## Test analyze_image_structured:

### Define file paths:

In [ ]:
jpg_data_path = os.path.join(script_dir, '..', 'test_llm_input/data_jpg')

In [ ]:
example_file = os.listdir(jpg_data_path)[0]
example_path = os.path.join(jpg_data_path, example_file)
example_path

### Define prompt function:

In [ ]:
def create_analysis_prompt_basic():
    """Create the structured prompt for image analysis."""
    return """
    Analyze this image and return ONLY a Python dictionary in exactly this format:
    
    {
        'image_is_photography': X,  # True if the image is a photography, False otherwise
        'additional_comments': '' # Any additional observations or empty string if none
    }
    
    Replace X with True (image is a photography) or False (image is not a photography).
    Return ONLY the dictionary, no other text.
    Your answer MUST have the exact structue of the dictionary described above (all keys MUST be present). 
    If you cannot answer the question in the way implied by this structure, enter 'None' as value and offer 
    your answer and explanations under 'additional_comments'.
    """

In [ ]:
def create_analysis_prompt_yes_no_basic():
    """Create the structured prompt for image analysis."""
    return """
    Is this image a photography or is it something else?

    Please, answwer yes if it is a photography, answer no if it is something else.
    
    """

### Execute function:

In [ ]:
output = llm_o.analyze_image_structured(example_path, create_analysis_prompt_basic)
output

In [ ]:
output = llm_o.analyze_image_structured(example_path, create_analysis_prompt_yes_no_basic)
output

In [ ]:
type(output.keys())

In [ ]:
list(output.keys())

In [ ]:
import pandas as pd

In [ ]:
test_frame = pd.DataFrame({'image_id': [0, 1, 2, 3], 'col1': [1, None, 3, 2], 'col2': [4, None, 3, 1]})
test_frame

In [ ]:
def add_pred_values(idx, labels_results, columns, values_to_add):

    """Function to manually add list of values to results data frame, in case
    structural anomalies in the llm response require manual processing
    of the data."""
    selection_bools = labels_results.image_id == idx
    
    labels_results.loc[selection_bools, columns] = values_to_add

In [ ]:
add_pred_values(1, test_frame, ['col1', 'col2'], [4, 5])

In [ ]:
test_frame

In [ ]:
type(test_frame)

In [ ]:
# Image identifiers defining which parts of the dictionary shall be processed:
image_ids = ['001', '002', '003', '004', '005']

# Input dictionary: 
image_descr = {}
image_descr['001'] = {'image_is_photography': True, 'additional_comments': ''}
image_descr['002'] = {'raw_response': 'I am not sure if the image is a photography. I think {maybe yes}'}
image_descr['003'] = {'raw_response': 'Haha, no structure at all! {alala}'}
image_descr['004'] = {'image_is_photography': False, 'additional_comments': 'The image looks like a painting.'}
image_descr['005'] = {'image_is_photography': True, 'additional_comments': ''}

# Dictionary keys that each entry of image_descr is expected to have if the entry is itself a dictionary:
keys_expected = ['image_is_photography', 'additional_comments']

# Variable of interest (ideally boolean values) that should be translated into one-hot-encoding:
response_variable = 'image_is_photography'

In [ ]:
image_descr = {}
image_descr['001'] = {'image_is_photography': True, 'additional_comments': ''}
image_descr['002'] = {'raw_response': 'I am not sure if the image is a photography. I think {maybe yes}'}
image_descr['003'] = {'raw_response': 'Haha, no structure at all! {alala}'}
image_descr['004'] = {'image_is_photography': False, 'additional_comments': 'The image looks like a painting.'}
image_descr['005'] = {'image_is_photography': True, 'additional_comments': ''}


In [ ]:
image_ids = ['001', '002', '003', '004', '005']
keys_expected = ['image_is_photography', 'additional_comments']
response_variable = 'image_is_photography'

output = llm_o.extract_vals_from_response_dict(image_ids, image_descr, keys_expected, response_variable)
output

In [ ]:
img_ids, response_values, img_ids_closer_inspection = output
print(img_ids)
print(response_values)
print(img_ids_closer_inspection)

In [ ]:
img_ids

In [ ]:
response_values

In [ ]:
img_ids_closer_inspection

In [ ]:
image_descr = {}
image_descr['001'] = {'image_is_photography': True, 'additional_comments': ''}
image_descr['002'] = {'raw_response': 'I am not sure if the image is a photography. I think {maybe yes}'}
image_descr['003'] = {'raw_response': 'Haha, no structure at all! {}'}
image_descr['004'] = {'Hey, no structure! What do you do now?'}
image_descr['005'] = {"What even is a dictionary? {Am I a dictionary?}"}

In [ ]:
# Image identifiers defining which parts of the dictionary shall be processed:

#image_ids = ['001', '002'] # These entries are processed successfully. The current function can handle this.
#image_ids = ['003'] # Yields an error message. The current function cannot handle this.
#image_ids = ['004'] # Yields an error message. The current function cannot handle this.
image_ids = ['005'] # Yields an error message. The current function cannot handle this.

# input dictionary:

image_descr = {}
image_descr['001'] = {'image_is_photography': True, 'additional_comments': ''}
image_descr['002'] = {'raw_response': 'I am not sure if the image is a photography. I think {maybe yes}'}
image_descr['003'] = {'raw_response': 'Haha, no structure at all! {}'}
image_descr['004'] = {'Hey, no structure! What do you do now?'}
image_descr['005'] = {"What even is a dictionary? {Am I a dictionary?}"}

# Dictionary keys that each entry of image_descr is expected to have if the entry is itself a dictionary:

keys_expected = ['image_is_photography', 'additional_comments']

# Variable of interest (ideally boolean values) that should be translated into one-hot-encoding:

response_variable = 'image_is_photography'

output = llm_o.extract_vals_from_response_dict(image_ids, image_descr, keys_expected, response_variable)
output

In [ ]:
def convert_boolean_to_encoding(bool_value):
    """
    Convert boolean value to one-hot encoding (1 or 0).
    
    Args:
        bool_value: The value to convert (should be boolean)
    
    Returns:
        1 if True, 0 if False, None if conversion fails
    """
    try:
        if isinstance(bool_value, bool):
            return int(bool_value)
        # Handle string representations of booleans
        elif isinstance(bool_value, str):
            if bool_value.lower() in ['true', '1', 'yes']:
                return 1
            elif bool_value.lower() in ['false', '0', 'no']:
                return 0
        # Handle numeric representations
        elif isinstance(bool_value, (int, float)):
            return 1 if bool_value else 0
    except Exception:
        pass
    
    return None


def extract_from_structured_dict(img_pred, keys_list_expected, response_variable):
    """
    Extract and convert response variable from a properly structured dictionary.
    
    Args:
        img_pred: Dictionary containing the response
        keys_list_expected: Expected keys in the dictionary
        response_variable: Key for the variable of interest
    
    Returns:
        Encoded value (0 or 1) or None if extraction fails
    """
    try:
        # Check if it's a dictionary
        if not isinstance(img_pred, dict):
            return None
            
        # Get keys and check structure
        keys_list = list(img_pred.keys())
        if sorted(keys_list_expected) != sorted(keys_list):
            return None
        
        # Extract and convert the response variable
        if response_variable not in img_pred:
            return None
            
        bool_value = img_pred[response_variable]
        return convert_boolean_to_encoding(bool_value)
        
    except Exception:
        return None


def extract_from_raw_response(img_pred, response_variable):
    """
    Extract and convert response variable from raw_response text.
    
    Args:
        img_pred: Dictionary containing 'raw_response' key
        response_variable: Key for the variable of interest
    
    Returns:
        Encoded value (0 or 1) or None if extraction fails
    """
    try:
        # Check if it's a dictionary and has raw_response key
        if not isinstance(img_pred, dict):
            return None
            
        if 'raw_response' not in img_pred:
            return None
            
        response_text = img_pred['raw_response']
        if not isinstance(response_text, str):
            return None
        
        # Use the existing parse_response_to_dict function
        success_bool, parsed_dict = parse_response_to_dict(response_text)
        
        # Check if parsing was successful and has our variable
        if not success_bool or parsed_dict is None:
            return None
            
        if not isinstance(parsed_dict, dict) or response_variable not in parsed_dict:
            return None
            
        bool_value = parsed_dict[response_variable]
        return convert_boolean_to_encoding(bool_value)
        
    except Exception:
        return None


def process_single_entry(img_id, img_pred, keys_list_expected, response_variable):
    """
    Process a single entry from the image description dictionary.
    
    Args:
        img_id: Image identifier (for logging)
        img_pred: The entry to process
        keys_list_expected: Expected dictionary keys
        response_variable: Key for the variable of interest
    
    Returns:
        Tuple of (encoded_value, needs_inspection)
        - encoded_value: 0, 1, or None
        - needs_inspection: True if processing failed
    """
    print(f"\n{img_id}")
    
    # Strategy 1: Try structured dictionary extraction
    try:
        encoded_value = extract_from_structured_dict(img_pred, keys_list_expected, response_variable)
        if encoded_value is not None:
            print("both conditions true")
            return encoded_value, False
    except Exception:
        pass
    
    # Strategy 2: Try raw_response extraction
    try:
        encoded_value = extract_from_raw_response(img_pred, response_variable)
        if encoded_value is not None:
            print("\nelif true")
            print("raw_repsonse_dict:")
            print(img_id)
            print("success_bool:")
            print(True)
            return encoded_value, False
    except Exception:
        pass
    
    # Strategy 3: Failed - needs manual inspection
    print("\nno structure at all:")
    print(img_id)
    
    return None, True


def extract_vals_from_response_dict(img_ids, image_descr, keys_list_expected, response_variable):
    """
    Function to extract values from response dictionary provided by an llm.

    input:
    - img_ids: list of image identifiers defining which entries of the 
    input dictionary should be processed.
    - image_descr: Dictionary where each entry is the analysis result referring to
    one image. Ideally, each entry is itself a dictionary, but there can be exceptions
    where an entry is just a character string.
    - keys_list_expected: List of dictionary keys that each entry of image_descr
    ideally should have if the entry is itself a dictionary.
    - response_variable: The dictionary key referring to the variable of interest, which must
    be a boolean. Each entry of image_descr should have this key if the entry is itself a dictionary.
    This boolean values are translated into one-hot-encoding by this function.

    output:
    - img_ids: the same list of image identifiers that is in the input.
    - response_values: A list of integers (either 1 or 0) or None (when the input could not be 
    successfully processed), where each integer value is the one-hot-encoding of the boolean value in 
    one entry of image_descr. In the image_descr entry this boolean value is referred to
    as response_variable (see input). In cases where the value of the response_variable
    could not be extraced and processed, the value of response_values is None.
    - img_ids_closer_inspection: List of image identifiers referring to those
    entries of image_descr that did not have the desired structure or keys and
    that therefore could not be successfully processed. 
    """
    # Input validation
    if not isinstance(img_ids, list) or not img_ids:
        raise ValueError("img_ids must be a non-empty list")
    
    if not isinstance(image_descr, dict):
        raise ValueError("image_descr must be a dictionary")
    
    # Check that all img_ids exist in image_descr
    missing_ids = [img_id for img_id in img_ids if img_id not in image_descr]
    if missing_ids:
        raise KeyError(f"Image IDs not found in image_descr: {missing_ids}")
    
    # Initialize output lists
    response_values = []
    img_ids_closer_inspection = []
    iter_count = 0
    
    # Process each image ID using try/except approach
    for img_id in img_ids:
        try:
            # Get response from LLM for image id in question
            img_pred = image_descr[img_id]
            
            # Process the entry using modular approach
            encoded_value, needs_inspection = process_single_entry(
                img_id, img_pred, keys_list_expected, response_variable
            )
            
            response_values.append(encoded_value)
            
            if needs_inspection:
                img_ids_closer_inspection.append(img_id)
                
        except KeyError:
            # This shouldn't happen due to validation above, but just in case
            print(f"\nKeyError: {img_id} not found in image_descr")
            response_values.append(None)
            img_ids_closer_inspection.append(img_id)
            
        except Exception as e:
            # Catch any other unexpected errors
            print(f"\nUnexpected error processing {img_id}: {e}")
            response_values.append(None)
            img_ids_closer_inspection.append(img_id)
        
        iter_count += 1
    
    # Verify output consistency
    assert len(img_ids) == len(response_values), "Output lists length mismatch"
    
    return img_ids, response_values, img_ids_closer_inspection

In [ ]:
# Image identifiers defining which parts of the dictionary shall be processed:

#image_ids = ['001', '002', '003'] # These entries are processed successfully. The current function can handle this.
image_ids = ['003'] # Yields an error message. The current function cannot handle this.
#image_ids = ['004'] # Yields an error message. The current function cannot handle this.
#image_ids = ['005'] # Yields an error message. The current function cannot handle this.

# input dictionary:

image_descr = {}
image_descr['001'] = {'image_is_photography': True, 'additional_comments': ''}
image_descr['002'] = {'raw_response': 'I am not sure if the image is a photography. I think {maybe yes}'}
image_descr['003'] = {'raw_response': 'Haha, no structure at all! {}'}
image_descr['004'] = {'Hey, no structure! What do you do now?'}
image_descr['005'] = {"What even is a dictionary? {Am I a dictionary?}"}

# Dictionary keys that each entry of image_descr is expected to have if the entry is itself a dictionary:

keys_expected = ['image_is_photography', 'additional_comments']

# Variable of interest (ideally boolean values) that should be translated into one-hot-encoding:

response_variable = 'image_is_photography'

output = extract_vals_from_response_dict(image_ids, image_descr, keys_expected, response_variable)
output

In [ ]:
img_ids, response_values, img_ids_closer_inspection = output
print(img_ids)
print(response_values)
print(img_ids_closer_inspection)

In [ ]:



img_id = '003'
img_pred = image_descr['003']
keys_list_expected = keys_expected

In [ ]:
process_single_entry(img_id, img_pred, keys_list_expected, response_variable)

In [ ]:
extract_from_structured_dict(img_pred, keys_list_expected, response_variable)

In [ ]:
img_id = '004'
img_pred = image_descr['004']

In [ ]:
img_pred

In [ ]:
response_variable

In [ ]:
#img_pred = {'raw_response': 'valid dict text'}
img_pred = {'raw_response': "{'image_is_photography': True, 'additional_comments': 'no comment'}"}
#img_pred = {'raw_response': "OK, I don't like dictionaries but here is one: {'image_is_photography': False, 'additional_comments': ''}"}
response_variable = 'image_is_photography'
output = extract_from_raw_response(img_pred, response_variable)
output

In [ ]:
llm_o.parse_response_to_dict("OK, I don't like dictionaries but here is one: {'image_is_photography': False, 'additional_comments': ''}")

In [ ]:
output = process_single_entry(img_id, img_pred, keys_list_expected, response_variable)
output

In [ ]:
if 345345:
    print('yes')

In [ ]:
345345 == True

In [ ]:
extract_from_raw_response(img_pred, response_variable)

In [ ]:
# Test each step individually
img_pred = {'raw_response': "{'image_is_photography': True, 'additional_comments': 'no comment'}"}
response_variable = 'image_is_photography'

print("Step 1 - Input validation:")
print("Is dict?", isinstance(img_pred, dict))
print("Has raw_response?", 'raw_response' in img_pred)

print("\nStep 2 - Extract raw_response:")
response_text = img_pred['raw_response']
print("Response text:", response_text)
print("Is string?", isinstance(response_text, str))

print("\nStep 3 - Parse with your function:")
success_bool, parsed_dict = llm_o.parse_response_to_dict(response_text)
print("Parse success:", success_bool)
print("Parsed dict:", parsed_dict)

print("\nStep 4 - Check parsed result:")
if parsed_dict:
    print("Parsed dict type:", type(parsed_dict))
    print("Is dict?", isinstance(parsed_dict, dict))
    print("Has response_variable?", response_variable in parsed_dict)
    if response_variable in parsed_dict:
        print("Response variable value:", parsed_dict[response_variable])

In [ ]:
# Test the final conversion step
bool_value = True  # This is what we got from the parsed dict
print("Bool value:", bool_value)
print("Bool value type:", type(bool_value))

# Test the conversion function directly
encoding_result = convert_boolean_to_encoding(bool_value)
print("Encoding result:", encoding_result)
print("Encoding result type:", type(encoding_result))

# Test the full extract_from_raw_response function with debug prints
print("\n--- Testing full function ---")
img_pred = {'raw_response': "{'image_is_photography': True, 'additional_comments': 'no comment'}"}
response_variable = 'image_is_photography'

result = extract_from_raw_response(img_pred, response_variable)
print("Final result:", result)
print("Final result type:", type(result))


In [ ]:
# Debug the exact conditions in extract_from_raw_response
img_pred = {'raw_response': "{'image_is_photography': True, 'additional_comments': 'no comment'}"}
response_variable = 'image_is_photography'

response_text = img_pred['raw_response']
success_bool, parsed_dict = llm_o.parse_response_to_dict(response_text)

print("success_bool:", success_bool)
print("parsed_dict:", parsed_dict)
print("parsed_dict is None:", parsed_dict is None)
print("isinstance(parsed_dict, dict):", isinstance(parsed_dict, dict))
print("response_variable in parsed_dict:", response_variable in parsed_dict)

# Check the specific condition that might be failing:
condition1 = not success_bool
condition2 = parsed_dict is None
condition3 = not isinstance(parsed_dict, dict)
condition4 = response_variable not in parsed_dict

print("\nCondition checks:")
print("not success_bool:", condition1)
print("parsed_dict is None:", condition2)  
print("not isinstance(parsed_dict, dict):", condition3)
print("response_variable not in parsed_dict:", condition4)

# The combined condition that causes return None:
failing_condition = condition1 or condition2 or condition3 or condition4
print("Should return None?", failing_condition)

In [ ]:
structured_text = "{'image_is_photography': True, 'additional_comments': 'no comment'}"
semi_structured_text = "OK, I don't like dictionaries but here is one: {'image_is_photography': False, 'additional_comments': ''}"
lacking_text = "{'image_is_photography': True}"

In [ ]:
llm_o.parse_response_to_dict(structured_text)

In [ ]:
llm_o.parse_response_to_dict(semi_structured_text)

In [ ]:
llm_o.parse_response_to_dict(lacking_text)

In [ ]:
def extract_from_raw_response(img_pred, response_variable):
    """
    Extract and convert response variable from raw_response text.
    
    Args:
        img_pred: Dictionary containing 'raw_response' key
        response_variable: Key for the variable of interest
    
    Returns:
        Encoded value (0 or 1) or None if extraction fails
    """
    try:
        #print('OK, I try.')
        # Check if it's a dictionary and has raw_response key
        if not isinstance(img_pred, dict):
            #print('img_pred data type:')
            #print(type(img_pred))
            return None
        #print('still here 1.')
        if 'raw_response' not in img_pred:
            #print('raw_response in img_pred:')
            #print('raw_response' in img_pred)
            return None
        #print('still here 2.')
        response_text = img_pred['raw_response']
        #print(response_text)
        if not isinstance(response_text, str):
            #print(type(response_text))
            #print(isinstance(response_text, str))
            return None
        
        # Use the existing parse_response_to_dict function
        success_bool, parsed_dict = llm_o.parse_response_to_dict(response_text)
        #print('\n')
        #print('intermediate result from parse_response_to_dict:')
        #print(success_bool)
        #print(parsed_dict)
        
        # Check if parsing was successful and has our variable
        if not success_bool or parsed_dict is None:
            #print('\n')
            #print('Either success_bool is false or parsed_dict is None:')
            #print('success_bool:')
            #print(success_bool)
            #print('parsed_dict:')
            #print(parsed_dict)
            return None
            
        if not isinstance(parsed_dict, dict) or response_variable not in parsed_dict:
            #print('\n')
            #print('parsed_dict data type:')
            #print(isinstance(parsed_dict, dict))
            #print('response_variable in parsed_dict:')
            #print(response_variable not in parsed_dict)
            return None
            
        bool_value = parsed_dict[response_variable]
        #print('\n')
        #print('bool_value:')
        #print(bool_value)
        #print(type(bool_value))
        return convert_boolean_to_encoding(bool_value)
        
    except Exception:
        return None
        

In [ ]:
structured_text

In [ ]:
#img_pred = {'raw_response': 'valid dict text'}
img_pred = {'raw_response': structured_text}
#img_pred = {'raw_response': semi_structured_text}
response_variable = 'image_is_photography'
output = llm_o.extract_from_raw_response(img_pred, response_variable)
output

In [ ]:
image_ids = [f'{i:03d}' for i in range(1, 9)]
with_person = [0] * 6 + [1] * 2
is_photo = [0] * 4 + [1] * 4
is_photo_pred = [0] * 2 + [1] * 2 + [0] * 2 + [1] * 2

In [ ]:
labels_results = pd.DataFrame({'image_id': image_ids, 'with_person': with_person, 
                 'is_photo': is_photo, 'is_photo_pred': is_photo_pred})
labels_results

In [ ]:
label_name = 'is_photo'
prediction_name = 'is_photo_pred'

In [ ]:
import source.llm_output as llm_o

In [ ]:
subsets_and_metrics = llm_o.get_classification_subsets_metrics(labels_results, label_name, prediction_name)
positives, negatives, true_positives, true_negatives, \
false_negatives, false_positives, sensitivity, specificity = subsets_and_metrics


In [ ]:
false_negatives

In [ ]:
false_positives

In [ ]:
sensitivity

In [ ]:
specificity 

In [ ]:
type(false_positives)

In [ ]:
type(false_positives.is_photo)

In [ ]:
type(list(false_positives.is_photo))

In [ ]:
sum(list(false_positives.is_photo))

In [ ]:
sum(list(false_positives.is_photo_pred))

In [ ]:
sum(list(true_positives.is_photo))

In [ ]:
sum(list(true_positives.is_photo_pred))

In [ ]:
pos_bools = labels_results.is_photo == 1
neg_bools = labels_results.is_photo == 0
pred_pos_bools = labels_results.is_photo_pred == 1
pred_neg_bools = labels_results.is_photo_pred == 0

labels_results[pos_bools & pred_pos_bools]

In [ ]:
labels_results[pos_bools & pred_pos_bools].shape[0]

In [ ]:
labels_results[pos_bools & pred_neg_bools].shape[0]

In [ ]:
labels_results[neg_bools & pred_neg_bools].shape[0]

In [ ]:
labels_results[neg_bools & pred_pos_bools].shape[0]